<a href="https://colab.research.google.com/github/Bryan-eng-lng/AI-Translatify/blob/main/AmazonReview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install kaggle

In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [8]:
!kaggle datasets download -d snap/amazon-fine-food-reviews


Dataset URL: https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews
License(s): CC0-1.0
 59% 143M/242M [00:00<00:00, 1.50GB/s]
100% 242M/242M [00:00<00:00, 932MB/s] 


In [9]:
!unzip amazon-fine-food-reviews.zip


Archive:  amazon-fine-food-reviews.zip
  inflating: Reviews.csv             
  inflating: database.sqlite         
  inflating: hashes.txt              


Lets Analyse the data

In [10]:
import pandas as pd

df = pd.read_csv("/content/Reviews.csv")


In [11]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [12]:
df.shape

(568454, 10)

In [13]:
df = df.drop(['Id','ProductId','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator','Time','Summary'],axis=1)

In [14]:
df.head()

,Score,Text
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...


In [15]:
# remove neutral reviews
df = df[df['Score'] != 3]

# create sentiment label
df['sentiment'] = df['Score'].apply(lambda x: 1 if x > 3 else 0)


In [16]:
df.head()

,Score,Text,sentiment
0,5,I have bought several of the Vitality canned d...,1
1,1,Product arrived labeled as Jumbo Salted Peanut...,0
2,4,This is a confection that has been around a fe...,1
3,2,If you are looking for the secret ingredient i...,0
4,5,Great taffy at a great price. There was a wid...,1


In [17]:
df = df.drop(columns=['Score'])

In [18]:
df.head()

,Text,sentiment
0,I have bought several of the Vitality canned d...,1
1,Product arrived labeled as Jumbo Salted Peanut...,0
2,This is a confection that has been around a fe...,1
3,If you are looking for the secret ingredient i...,0
4,Great taffy at a great price. There was a wid...,1


In [19]:
df.sentiment.value_counts()

,count
sentiment,
1,443777
0,82037


4,5 = Positive review = 1

1,2 = Negative review = 0

In [20]:
!pip install transformers datasets torch

In [21]:
import torch
from transformers import BertTokenizer , BertForSequenceClassification
from torch.utils.data import Dataset , DataLoader
from sklearn.model_selection import train_test_split

In [22]:
df = df.sample(10000 , random_state=42)

In [23]:
X = list(df['Text'])
y = list(df['sentiment'])

In [24]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.2 , random_state=42)

In [25]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [40]:
class ReviewDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }


In [41]:
train_dataset = ReviewDataset(X_train , y_train , tokenizer)

test_dataset = ReviewDataset(X_test , y_test , tokenizer)

In [42]:
train_loader = DataLoader(train_dataset , batch_size=16 , shuffle=True)
test_loader = DataLoader(test_dataset , batch_size=16)

In [43]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [44]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

model.train()

for epoch in range(2):
    print(f"Epoch {epoch+1}")
    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        print("Loss:", loss.item())

Epoch 1
Loss: 0.7683559060096741
Loss: 0.6276423931121826
Loss: 0.5473924875259399
Loss: 0.5983481407165527
Loss: 0.5637490749359131
Loss: 0.446696400642395
Loss: 0.5060010552406311
Loss: 0.4526922106742859
Loss: 0.4772016108036041
Loss: 0.4065523147583008
Loss: 0.5170137882232666
Loss: 0.3627830147743225
Loss: 0.36373597383499146
Loss: 0.48507535457611084
Loss: 0.5033546090126038
Loss: 0.24825841188430786
Loss: 0.23948629200458527
Loss: 0.6343340277671814
Loss: 0.3597588539123535
Loss: 0.6337952613830566
Loss: 0.2230456918478012
Loss: 0.637770414352417
Loss: 0.2748364210128784
Loss: 0.35930272936820984
Loss: 0.33969444036483765
Loss: 0.7412413358688354
Loss: 0.6190105080604553
Loss: 0.5075807571411133
Loss: 0.2618221342563629
Loss: 0.3541192412376404
Loss: 0.3718143105506897
Loss: 0.5712623596191406
Loss: 0.3002776503562927
Loss: 0.36576950550079346
Loss: 0.38329070806503296
Loss: 0.2713863253593445
Loss: 0.3771354556083679
Loss: 0.4877047538757324
Loss: 0.46188247203826904
Loss: 0.34

In [46]:
model.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [48]:
def predict_review_sentiment(review, model, tokenizer, max_len=128):
    model.eval()

    encoding = tokenizer(
        review,
        truncation=True,
        padding='max_length',
        max_length=max_len,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        probs = torch.softmax(outputs.logits, dim=1)
        prediction = torch.argmax(probs, dim=1).item()

    return prediction, probs


In [50]:
user_review = input("Enter Amazon food review: ")

pred, probs = predict_review_sentiment(user_review, model, tokenizer)

if pred == 1:
    print("Positive Review")
else:
    print("Negative Review")

print("Confidence:", probs)


Enter Amazon food review: Worst product ever. Tastes horrible and made me sick.
Negative Review
Confidence: tensor([[0.9934, 0.0066]], device='cuda:0')
